Install necessary packages

In [ ]:
#!pip3 install torch transformers spacy scikit-learn nltk pandas
#!python3 -m spacy download en_core_web_md
#nltk.download('punkt')

In [2]:
import nltk
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import string
import json
import string
import random
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torch import nn
import main


Split Training/Testing/Validation

In [ ]:
temp = [{
    "message": "I'm getting a 'syntax error' in my code.",
    "response": "Ah, syntax errors, the classics!"
},{
    "message": "I'm getting a 'syntax error' in my code.",
    "response": "No worries, we can fix that."
},{
    "message": "I'm getting a 'syntax error' in my code.",
    "response": "Could you paste the troublesome code here?"
}
       ]

In [ ]:
json_data = json.dumps(temp, indent=4)

In [3]:
#reverse=True
temp=main.calculate_prompts("input.json")

loading parameters


/home/user/bokboard/main.py:47: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_scores = [nlp(w).similarity(word) for w in str(nlp.vocab)]


complete


In [4]:
main.save_json(temp)

Results saved to file


In [ ]:
#reverse=False
main.calculate_prompts("input.json")

In [ ]:
# Load a pre-trained spaCy model with word embeddings (e.g., en_core_web_md)
nlp = spacy.load("en_core_web_md")

# Read input data from a JSON file
with open("input.json", "r") as input_file:
    input_data = json.load(input_file)

#List of stopwords
stopwords = set(nlp.Defaults.stop_words)

# Initialize a dictionary to store results
results = []

#Function to calculate uniqueness score
def calculate_word_uniqueness(word):
    word_vector = word.vector
    similarity_scores = [nlp(w).similarity(word) for w in str(nlp.vocab)]
    uniqueness_score = sum(similarity_scores) / len(similarity_scores)
    return uniqueness_score

# Calculate prompt words for each section using word subtraction
for i, section in enumerate(input_data):
    message = section["message"]
    response = section["response"]

    # Tokenize the messages using spaCy
    message_tokens = nlp(message.lower())
    response_tokens = nlp(response.lower())

    # Find prompt words using word subtraction
    prompt_words = list(set([token.text for token in response_tokens]) - set([token.text for token in message_tokens]) - set([token.text for token in response_tokens if "'" in token.text]))

    # Initialize variables to keep track of the most unique word and its score for this prompt
    most_unique_word = None
    max_uniqueness_score = -1  # Initialize with a very low value

    # Remove punctuation from the list of prompt words using the string module
    #cleaned_prompt_words = [word.translate(str.maketrans('', '', string.punctuation)) for word in prompt_words]
    print(prompt_words)
    #print(cleaned_prompt_words)
    # Calculate the uniqueness score (cosine similarity) for each non-stopword word in the list
    for word in prompt_words:
        # Check if the word is not empty after removing punctuation
        if word.strip() and word not in stopwords:
            uniqueness_score = calculate_word_uniqueness(nlp(word))

            # Check if this word is more unique than the previous most unique word
            if uniqueness_score > max_uniqueness_score:
                max_uniqueness_score = uniqueness_score
                most_unique_word = word

    # Store the result for this section in the dictionary
    results.append({
        "message": message,
        "response": response,
        "prompt_words": prompt_words,
        "prompt": most_unique_word,
        "uniqueness_score": float(max_uniqueness_score),
        "system_prompt": "You are a helpful assistant. Generate a response to the following conversation:"
    })

# Save the results as a JSON file
with open("uniqueness_scores.json", "w") as json_file:
    json.dump(results, json_file, indent=4)

print("Results saved to 'uniqueness_scores.json'")

In [ ]:
# Load a pre-trained spaCy model with word embeddings (e.g., en_core_web_md)
nlp = spacy.load("en_core_web_md")

# Read input data from a JSON file
with open("uniqueness_scores.json", "r") as input_file:
    input_data = json.load(input_file)

# Shuffle the data randomly to ensure randomness in the split
random.shuffle(input_data)

# Define the proportions for the split (adjust as needed)
train_split = 0.7  # 70% for training
test_split = 0.15  # 15% for testing
validation_split = 0.15  # 15% for validation

# Calculate the split sizes based on the proportions
total_samples = len(input_data)
train_size = int(train_split * total_samples)
test_size = int(test_split * total_samples)

# Split the data
train_data = input_data[:train_size]
test_data = input_data[train_size:train_size + test_size]
validation_data = input_data[train_size + test_size:]

# Save the split datasets as separate JSON files
with open("train_data.json", "w") as train_file:
    json.dump(train_data, train_file, indent=4)

with open("test_data.json", "w") as test_file:
    json.dump(test_data, test_file, indent=4)

with open("validation_data.json", "w") as validation_file:
    json.dump(validation_data, validation_file, indent=4)

print("Data split into training, testing, and validation sets.")

Train

In [12]:
def generate_input_texts(training_data):
    # Initialize lists to store combined input text
    input_texts = []
    
    # Combine the input data into input_texts
    for entry in training_data:
        message = entry["message"]
        response_message = entry["response"]
        system_prompt = "You are a helpful assistant. Generate a response to the following message using the prompt."
        prompt = ' '.join(entry["prompt"])
        input_text = f"{system_prompt}\nmessage: {message}\nprompt: {prompt}\nresponse{response_message}\n"
        input_texts.append(input_text)
    return(input_texts)

In [13]:
# Load the training data from the JSON file
with open("train_data.json", "r") as json_file:
    training_data = json.load(json_file)

input_texts = generate_input_texts(training_data)

# Initialize the falcon-7b-sharded model and tokenizer
model_name = "vilsonrodrigues/falcon-7b-sharded"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

#for input_text in input_texts:
#    tokens = tokenizer.encode(input_text, return_tensors="pt")
#    print("Input Text:", input_text)
#    print("Tokens:", tokens)

# # Tokenize the input texts without subword splitting
# input_tokens = [tokenizer.tokenize(text) for text in input_texts]

# # Convert tokens to input IDs
# input_ids = [tokenizer.convert_tokens_to_ids(tokens) for tokens in input_tokens]


# Tokenize the input texts
encoded_input = tokenizer(
    input_texts,
    return_tensors="pt",
    padding="max_length",
    max_length=512  # Adjust max_length as needed
)

def truncate(encoded_input):
  encoded_input_trc={}
  for k,v in encoded_input.items():
      v_truncated = v[:,:512]
      encoded_input_trc[k]=v_truncated
  return encoded_input_trc

input_ids = truncate(encoded_input)

# Define a custom dataset for training
class TextDataset(Dataset):
    def __init__(self, input_ids):
        self.input_ids = input_ids

    def __len__(self):
        return len(self.input_ids["input_ids"])

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.input_ids.items()}

# Create a DataLoader for training
dataset = TextDataset(input_ids)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Training configuration
num_epochs = 5
learning_rate = 1e-4
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    total_loss = 0
    model.train()

    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}"):
        for key in batch:
            batch[key] = batch[key].to(device)

        optimizer.zero_grad()
        outputs = model(**batch, labels=batch["input_ids"])
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1} Loss: {total_loss / len(dataloader)}")

# Save the trained model
model.save_pretrained("trained_falcon_model")
tokenizer.save_pretrained("trained_falcon_model")

print("Training completed. Model saved as 'trained_falcon_model'.")


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Epoch 1:   0%|                                                                                    | 0/3 [00:00<?, ?it/s]


IndexError: index out of range in self

In [14]:
# Load the trained falcon-7b-sharded model and tokenizer
model_path = "trained_falcon_model"  # Replace with the path to your trained model
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Set the device for inference (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define a user prompt for text generation
user_prompt = "Please provide recommendations for places to visit in"

# Generate text based on the user prompt
generated_text = model.generate(
    input_ids=tokenizer.encode(user_prompt, return_tensors="pt").to(device),
    max_length=100,  # Adjust as needed for desired text length
    num_return_sequences=1,  # Number of sequences to generate
    temperature=0.7,  # Adjust for randomness (higher values make text more random)
    top_k=50,  # Adjust to control the diversity of generated text
    top_p=0.95,  # Adjust to control the diversity of generated text
    pad_token_id=50256,  # GPT-2 token for padding
    eos_token_id=50256,  # GPT-2 token for end of sequence
    bos_token_id=50256,  # GPT-2 token for beginning of sequence
)

# Decode and print the generated text
generated_text = tokenizer.decode(generated_text[0], skip_special_tokens=True)
print("Generated Text:")
print(generated_text)


OSError: trained_falcon_model is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
input_dim = 10
embedding_dim = 2
embedding = nn.Embedding(input_dim, embedding_dim)
err = False
if err:
    #Any input more than input_dim - 1, here input_dim = 10
    #Any input less than zero
    input_to_embed = torch.tensor([10])
else:
    input_to_embed = torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
embed = embedding(input_to_embed)
print(embed)